<a href="https://colab.research.google.com/github/MargaridaVitolo/Educ360/blob/main/Semana7_Teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import sqlite3
import pandas as pd

# Conectar banco de dados SQLite em memória
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
# Script SQL para criar e popular as tabelas
script_sql = """
DROP TABLE IF EXISTS vendas;
DROP TABLE IF EXISTS produtos;

CREATE TABLE produtos (
    id_produto INTEGER PRIMARY KEY,
    nome_produto TEXT,
    categoria TEXT,
    preco_unitario FLOAT
);

CREATE TABLE vendas (
    id_venda INTEGER PRIMARY KEY,
    id_produto INTEGER,
    quantidade INTEGER,
    data_venda DATE,
    desconto FLOAT,
    FOREIGN KEY (id_produto) REFERENCES produtos(id_produto)
);

INSERT INTO produtos (id_produto, nome_produto, categoria, preco_unitario) VALUES
(1, 'Notebook Dell', 'Informática', 3500.00),
(2, 'Mouse Logitech', 'Acessórios', 120.00),
(3, 'Monitor LG 24\"', 'Informática', 950.00),
(4, 'Teclado Mecânico', 'Acessórios', 420.00),
(5, 'Headset Gamer', 'Acessórios', 480.00),
(6, 'Impressora HP', 'Periféricos', 800.00),
(7, 'Smartphone Samsung', 'Telefonia', 2900.00),
(8, 'Cabo HDMI 2m', 'Acessórios', 40.00),
(9, 'Webcam Logitech', 'Acessórios', 310.00),
(10, 'Roteador TP-Link', 'Redes', 250.00),
(11, 'SSD Kingston 480GB', 'Armazenamento', 380.00),
(12, 'HD Externo 1TB', 'Armazenamento', 420.00),
(13, 'Tablet Lenovo', 'Informática', 1300.00),
(14, 'Monitor Samsung 27\"', 'Informática', NULL),
(15, 'Caixa de Som JBL', NULL, 550.00);

INSERT INTO vendas (id_venda, id_produto, quantidade, data_venda, desconto) VALUES
(1, 1, 3, '2025-10-01', 50.00),
(2, 2, 5, '2025-10-02', 0.00),
(3, 3, 2, '2025-10-03', NULL),
(4, 4, 1, '2025-10-04', 20.00),
(5, 5, 4, '2025-10-05', 15.00),
(6, 6, 2, '2025-10-06', 0.00),
(7, 7, 1, '2025-10-07', 200.00),
(8, 8, 10, '2025-10-08', NULL),
(9, 9, 3, '2025-10-09', 30.00),
(10, 10, 2, '2025-10-10', 10.00),
(11, 11, 5, '2025-10-11', 0.00),
(12, 12, 3, '2025-10-12', 25.00),
(13, 13, 1, '2025-10-13', 0.00),
(14, 14, 2, '2025-10-14', 50.00),
(15, 15, 6, '2025-10-15', NULL),
(16, 1, 1, '2025-10-16', 0.00),
(17, 3, 4, '2025-10-17', 30.00),
(18, 5, 2, '2025-10-18', NULL),
(19, 7, 2, '2025-10-19', 150.00),
(20, 8, NULL, '2025-10-20', 0.00),
(21, 9, 1, '2025-10-21', NULL),
(22, 10, 3, '2025-10-22', 0.00),
(23, 11, NULL, '2025-10-23', 20.00),
(24, 12, 2, '2025-10-24', 0.00),
(25, 13, 1, '2025-10-25', 10.00),
(26, 14, NULL, '2025-10-26', NULL),
(27, 15, 5, '2025-10-27', 0.00),
(28, NULL, 2, '2025-10-28', 0.00),
(29, 6, 1, '2025-10-29', 0.00),
(30, 2, 4, '2025-10-30', 10.00);
"""

# Executar criação e inserção
cursor.executescript(script_sql)
conn.commit()


In [4]:

# Explorar dados das tabelas
produtos_df = pd.read_sql_query('SELECT * FROM produtos', conn)
vendas_df = pd.read_sql_query('SELECT * FROM vendas', conn)

In [5]:
# Informações produtos
print("Produtos:")
print("Colunas:", produtos_df.columns.tolist())
print("Tipos:", produtos_df.dtypes.to_dict())
print("Quantidade de registros:", len(produtos_df))
print("Valores nulos:\n", produtos_df.isnull().sum())

Produtos:
Colunas: ['id_produto', 'nome_produto', 'categoria', 'preco_unitario']
Tipos: {'id_produto': dtype('int64'), 'nome_produto': dtype('O'), 'categoria': dtype('O'), 'preco_unitario': dtype('float64')}
Quantidade de registros: 15
Valores nulos:
 id_produto        0
nome_produto      0
categoria         1
preco_unitario    1
dtype: int64


In [6]:
print("\nVendas:")
print("Colunas:", vendas_df.columns.tolist())
print("Tipos:", vendas_df.dtypes.to_dict())
print("Quantidade de registros:", len(vendas_df))
print("Valores nulos:\n", vendas_df.isnull().sum())


Vendas:
Colunas: ['id_venda', 'id_produto', 'quantidade', 'data_venda', 'desconto']
Tipos: {'id_venda': dtype('int64'), 'id_produto': dtype('float64'), 'quantidade': dtype('float64'), 'data_venda': dtype('O'), 'desconto': dtype('float64')}
Quantidade de registros: 30
Valores nulos:
 id_venda      0
id_produto    1
quantidade    3
data_venda    0
desconto      6
dtype: int64


In [7]:
# Consulta unindo as tabelas e calculando valor_total
query = """
SELECT
    v.id_venda,
    p.nome_produto,
    p.categoria,
    p.preco_unitario,
    v.quantidade,
    v.desconto,
    v.data_venda,
    COALESCE(p.preco_unitario, 0) * COALESCE(v.quantidade, 0) - COALESCE(v.desconto, 0) AS valor_total
FROM vendas v
LEFT JOIN produtos p ON v.id_produto = p.id_produto
WHERE v.id_produto IS NOT NULL
ORDER BY v.id_venda
"""

resultado_df = pd.read_sql_query(query, conn)
print("\nConsulta com valor_total calculado:")
print(resultado_df.head(10))  # mostrar os 10 primeiros registros



Consulta com valor_total calculado:
   id_venda        nome_produto    categoria  preco_unitario  quantidade  \
0         1       Notebook Dell  Informática          3500.0         3.0   
1         2      Mouse Logitech   Acessórios           120.0         5.0   
2         3      Monitor LG 24"  Informática           950.0         2.0   
3         4    Teclado Mecânico   Acessórios           420.0         1.0   
4         5       Headset Gamer   Acessórios           480.0         4.0   
5         6       Impressora HP  Periféricos           800.0         2.0   
6         7  Smartphone Samsung    Telefonia          2900.0         1.0   
7         8        Cabo HDMI 2m   Acessórios            40.0        10.0   
8         9     Webcam Logitech   Acessórios           310.0         3.0   
9        10    Roteador TP-Link        Redes           250.0         2.0   

   desconto  data_venda  valor_total  
0      50.0  2025-10-01      10450.0  
1       0.0  2025-10-02        600.0  
2       N